In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
pd.set_option('display.max_columns', None)
import math
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer


import statsmodels.stats.stattools as stools
import statsmodels.stats as stats 
from statistics import mode 
import matplotlib.pyplot as plt

!pip install h2o
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.automl import H2OAutoML

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 177.0 MB 14 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=4607e7c12a0b15c5dc34aa73ca50697ce255a24964e130af31528b605164d664
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [ ]:


usinglist = ['ShortName','Ticker','Sector','Region','ImpliedRating','Spread','Tenor']


In [ ]:
# Import data
useful_rawdata = pd.read_csv('/content/drive/MyDrive/Practicum/Week10/Copy of Copy of Composite_CDS_Basic_Pricing_20220228.csv', header = 0)
equity_data = pd.read_csv('/content/drive/MyDrive/Practicum/Week10/Copy of Debt per share-2.csv', header = 0)

#data processing with custom methods

#Only keep Y in 'PrimaryCurve' and 'PrimaryCoupon'
useful_rawdata = useful_rawdata[(useful_rawdata['PrimaryCurve'] == 'Y' )& (useful_rawdata['PrimaryCoupon'] == 'Y') ]

#drop CCC and D
useful_rawdata = useful_rawdata[~(useful_rawdata['ImpliedRating'] == 'CCC') & ~(useful_rawdata['ImpliedRating'] == 'D')]

#drop sub senior
useful_rawdata = useful_rawdata[~(useful_rawdata['Tier'] == 'SUBLT2') & ~(useful_rawdata['Tier'] == 'SECOND') & ~(useful_rawdata['Tier'] == 'SNRLAC')]

#Aggregate region and sector as ECB's instruction

useful_rawdata = useful_rawdata[~(useful_rawdata['Sector'] == 'Government')] 

useful_rawdata['Region'] = useful_rawdata['Region'].apply(lambda x: 'North America' if x in ['N.Amer'] else x)
useful_rawdata['Region'] = useful_rawdata['Region'].apply(lambda x: 'Europe' if x in ['Europe','E.Eur'] else x)
useful_rawdata['Region'] = useful_rawdata['Region'].apply(lambda x: 'Asia' if x in ['Asia','India'] else x)
useful_rawdata['Region'] = useful_rawdata['Region'].apply(lambda x: 'others' if x not in ['Asia','North America','Europe'] else x) 

useful_rawdata.loc[useful_rawdata['Country']=='United States','Region']='United States'
useful_rawdata.loc[useful_rawdata['Country']=='Japan','Region']='Japan'
#Drop DocClause is MM or CR
useful_rawdata = useful_rawdata[~(useful_rawdata['DocClause'] == 'MM') & ~(useful_rawdata['DocClause'] == 'CR')]

#CDS spread normalization and times 100

useful_rawdata['Spread'] = useful_rawdata['ParSpread'] *10000 * 0.6/(1 - useful_rawdata['CDSRealRecovery'])

processed_data = useful_rawdata[usinglist]
processed_data.rename(columns={'ImpliedRating':'Rating'}, inplace=True)

#Aggrate two table together
agg_data = pd.merge(processed_data, equity_data, how = 'right', left_on='ShortName', right_on = 'Name')
agg_data = agg_data.drop(['ShortName','Ticker_x','Name','Ticker_y'], axis = 1 )

agg_data

,Sector,Region,Rating,Spread,Tenor,Unnamed: 0,Leverage,52week_high,52week_low,Volatility_1m,Volatility_2m,Volatility_3m,Volatility_4m,Volatility_6m,return_1m,return_2m,return_3m,return_4m,return_6m,industry,FinD,Division,Debt per share,Current Price
0,Basic Materials,United States,B,434.450862,5Y,0,192.7520,30.00,12.00,0.008243,0.006509,0.018077,0.024719,0.024151,0.038597,0.034965,0.106542,0.286957,0.409524,Non-bank,NaN,NaN,NaN,NaN
1,Basic Materials,United States,B,470.483077,15Y,0,192.7520,30.00,12.00,0.008243,0.006509,0.018077,0.024719,0.024151,0.038597,0.034965,0.106542,0.286957,0.409524,Non-bank,NaN,NaN,NaN,NaN
2,Basic Materials,United States,B,263.399538,3M,0,192.7520,30.00,12.00,0.008243,0.006509,0.018077,0.024719,0.024151,0.038597,0.034965,0.106542,0.286957,0.409524,Non-bank,NaN,NaN,NaN,NaN
3,Basic Materials,United States,B,503.351077,30Y,0,192.7520,30.00,12.00,0.008243,0.006509,0.018077,0.024719,0.024151,0.038597,0.034965,0.106542,0.286957,0.409524,Non-bank,NaN,NaN,NaN,NaN
4,Basic Materials,United States,B,289.577815,2Y,0,192.7520,30.00,12.00,0.008243,0.006509,0.018077,0.024719,0.024151,0.038597,0.034965,0.106542,0.286957,0.409524,Non-bank,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15876,Consumer Services,United States,A,70.540900,7Y,1475,-92695.2381,305.71,180.42,0.014675,0.013493,0.012303,0.013738,0.013158,0.072686,0.107857,0.270437,0.326225,0.351822,Non-bank,3.583400e+10,149797998.9,239.215478,306.13
15877,Consumer Services,United States,A,100.522000,20Y,1475,-92695.2381,305.71,180.42,0.014675,0.013493,0.012303,0.013738,0.013158,0.072686,0.107857,0.270437,0.326225,0.351822,Non-bank,3.583400e+10,149797998.9,239.215478,306.13
15878,Consumer Services,United States,A,24.815500,3Y,1475,-92695.2381,305.71,180.42,0.014675,0.013493,0.012303,0.013738,0.013158,0.072686,0.107857,0.270437,0.326225,0.351822,Non-bank,3.583400e+10,149797998.9,239.215478,306.13
15879,Consumer Services,United States,A,86.540600,10Y,1475,-92695.2381,305.71,180.42,0.014675,0.013493,0.012303,0.013738,0.013158,0.072686,0.107857,0.270437,0.326225,0.351822,Non-bank,3.583400e+10,149797998.9,239.215478,306.13


In [ ]:
class Dataset:
  # For new data input, we can do data pre-processing, choose best feature for equity and volatility, add E2C and splite tenor 
  label = 'Spread'
  tenor = ['1Y','2Y','3Y','4Y','5Y','7Y','10Y']
  basic_feature = ['Sector','Region','Rating']
  interaction_feature = []
  other_feature = ['E2C']
  er_list = ['return_1m','return_2m', 'return_3m', 'return_4m', 'return_6m']
  vol_list = ['Volatility_1m', 'Volatility_2m','Volatility_3m', 'Volatility_4m', 'Volatility_6m']

  #using as all feature name documentation 
  new_datalist = ['Sector','Region','Rating']

  data_export_path = '/content/drive/MyDrive/Practicum/Week10/Data/'


  def __init__(self, df):
    self.df_ = df
    self._tenor_split()
    


  def _tenor_split(self):
    for i in range(len(self.tenor)):
      self.__dict__['df'+ self.tenor[i]] = self.df_[self.df_['Tenor'] == self.tenor[i]]


  def _highest_feature_cor(self, name_list):
    # calculate correlation between label and different feature tenor, using mode one as best feature
    result_list = []
    name_list.append(self.label) 
    for i in range(len(self.tenor)):
      name = 'df' + self.tenor[i]
      df = self.__dict__[name]
      candidate = df[name_list]
      cor_matrix = candidate.corr()
      modify_matrix = cor_matrix.drop([self.label])
      modify_matrix = modify_matrix[self.label]

      max_one = modify_matrix.abs().idxmax()
      temp = max_one
      result_list.append(temp)
    return mode(result_list)
  
  def _E2C(self):
    for i in range(len(self.tenor)):
      name = self.tenor[i]
      df =  self.__dict__['df'+name]
      df['Debt per share'] = df['Debt per share'].apply(lambda x: np.nan if x == '#DIV/0!' else x)
      df['Debt per share'] = df['Debt per share'].dropna()
    
      df[['Debt per share','Current Price','Volatility_2m']] = df[['Debt per share','Current Price','Volatility_2m']].astype(float)
      fomula = 0.7*4/9*0.5*df['Debt per share']/(df['Current Price']+0.5*df['Debt per share'])*df['Volatility_2m'].apply(math.sqrt)
      df['E2C'] = fomula
  
  def _return_range(self):
    for i in range(len(self.tenor)):
      name = self.tenor[i]
      df =  self.__dict__['df'+name]
      df['return_range'] = (df['52week_high'] - df['52week_low'])/(df['52week_high'] + df['52week_low'])


  def choose_best_feature(self):

    self._E2C()
    self._return_range()

    eq_choice = self._highest_feature_cor(self.er_list)
    vol_choice = self._highest_feature_cor(self.vol_list)

    self.new_datalist.append(eq_choice)
    self.new_datalist.append(vol_choice)
    self.new_datalist.append('Leverage')
    self.new_datalist.append('E2C')
    self.new_datalist.append('return_range')
    self.new_datalist.append(self.label)
    
    # document interaction feature name
    self.interaction_feature.append(eq_choice)
    self.interaction_feature.append(vol_choice)
    self.interaction_feature.append('Leverage')
    self.interaction_feature.append('return_range')

    for i in range(len(self.tenor)):
      name = self.tenor[i]
      self.__dict__['df'+name] = self.__dict__['df'+name][self.new_datalist]
    
  
  def export(self, path):
      for i in range(len(self.tenor)):
        name = 'df' + self.tenor[i]
        temp = path + name+'.csv'
        self.__dict__[name].to_csv(temp, index=False)

In [ ]:
test = Dataset(agg_data)

In [ ]:
test.choose_best_feature()

In [ ]:
test.df5Y

,Sector,Region,Rating,return_2m,Volatility_1m,Leverage,E2C,return_range,Spread
0,Basic Materials,United States,B,0.034965,0.008243,192.752000,NaN,0.428571,434.450862
12,Consumer Services,Asia,BB,-0.017143,0.014564,247.349000,NaN,0.206349,269.860533
19,Energy,Europe,A,0.249078,0.018137,57.229000,0.011221,0.214278,43.088439
30,Energy,Europe,A,0.249078,0.017952,57.229000,0.011260,0.214278,43.028600
43,Telecommunications Services,United States,BBB,0.312500,0.561334,-200.243926,NaN,0.999425,108.045500
...,...,...,...,...,...,...,...,...,...
15817,Consumer Services,United States,BB,0.032102,0.025375,-3615.364904,0.020798,0.348940,175.040300
15829,Consumer Services,United States,BB,-0.104894,0.013831,-154.353278,0.006053,0.137639,146.770400
15843,Financials,United States,A,0.093818,0.020680,72.601945,0.008181,0.205680,59.588100
15856,Consumer Services,United States,AA,-0.069459,0.008615,-1183.118887,0.003823,0.109384,31.645900


In [ ]:
test.df5Y[(test.df5Y['Rating'] == 'B')| (test.df5Y['Region'] == 'Europe') | ]['Spread']

0        434.450862
19        43.088439
30        43.028600
52        67.959700
101      438.926300
            ...    
15695     50.922100
15708     66.382500
15720     44.550800
15732     95.377900
15758    428.553553
Name: Spread, Length: 452, dtype: float64

In [ ]:
()
class model:

  model_list = {
  "linear": "LinearRegression(fit_intercept=False)",
  'random_forest': 'H2ORandomForestEstimator',
  'autoML': 'H2OAutoML'
  }

  result_export_path = '/content/drive/MyDrive/Practicum/Week10/Result/'

  evaluation_matrix = ['rmse','train_R2', 'test_R2']

  RF_parameters = {'ntrees': [30, 50, 70, 90, 100],
          'max_depth': [5, 10, 15, 20 , 30, 40]}
  
  
  def __init__(self,data,feature,model_type):
    
    for i in range(len(data.tenor)):
      name = data.tenor[i]
      self.__dict__['df'+name] = data.__dict__['df'+name].copy()
      self.__dict__['df'+name] = self.__dict__['df'+name][feature]
    
    self.interaction_feature = self._list_same(feature, data.interaction_feature)
    self.basic_feature = self._list_same(feature, data.basic_feature)
    self.other_feature = self._list_same(feature, data.other_feature)
    self.feature_list = self.interaction_feature + self.basic_feature + self.other_feature
    self.tenor = data.tenor
    self.label = data.label
    
    self.model_type = model_type
    if self.model_type == "linear":
      self.model = LinearRegression(fit_intercept=False) 
    
    if self.model_type == 'random_forest':
      #initialize H2O
      h2o.init()
      self.model = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='RFR_grid',
                          hyper_params=self.RF_parameters)
    
    if self.model_type == 'autoML':
      h2o.init()
      self.model = H2OAutoML(max_models=20, seed=1,exclude_algos = ['StackedEnsemble'])




    self._get_velidation_list()

  
  def _data_process(self):
    if self.model_type == "linear":
      for i in range(len(self.tenor)):
        name = 'df' + self.tenor[i]
        df  = self.__dict__[name]
        df = df.dropna()
        df = self._outlier_process(df)
        df = self._min_max_scaling(df)
        df[self.label] = df[self.label].apply(np.log)
        self.__dict__[name] = df.copy()
    
    if self.model_type == "random_forest":
      for i in range(len(self.tenor)):
        name = 'df' + self.tenor[i]
        df  = self.__dict__[name]
        df = df.dropna()
        df = self._outlier_process(df)
        df[self.label] = df[self.label].apply(np.log)
        self.__dict__[name] = df.copy()
    if self.model_type == "autoML":
      for i in range(len(self.tenor)):
        name = 'df' + self.tenor[i]
        df  = self.__dict__[name]
        df = df.dropna()
        df = self._outlier_process(df)
        df[self.label] = df[self.label].apply(np.log)
        self.__dict__[name] = df.copy()

  
  def model_validation(self, cv_no, specific_tenor, spec):
    
    self.spec = spec
    #spec is the choice in basic feature that need to add interaction term
    self._data_process()

    if specific_tenor == 0:
      tenor = self.tenor
    else:
      tenor = [specific_tenor]

    if self.model_type == "linear":
      for i in range(len(tenor)):
        name = 'df' + tenor[i]
        df  = self.__dict__[name]

        sector_result = self._partial_validation(df,'Sector',cv_no)
        region_result = self._partial_validation(df,'Region',cv_no)
        rating_result = self._partial_validation(df,'Rating',cv_no)
        result = sector_result + region_result + rating_result
        

        self.__dict__['result'+name] = pd.DataFrame()
        self.__dict__['result'+name]['Name'] = self.name_list
        self.__dict__['result'+name][self.evaluation_matrix] = result

        marker_list = self.interaction_feature + self.other_feature
        marker = [x[0].upper() for x in marker_list]
        marker = '_'.join(marker)
        path = self.result_export_path+name+'_'+ marker+'_' +'result.csv'
        self.__dict__['result'+name].to_csv(path, index=False)
      
    if self.model_type == "random_forest":
      for i in range(len(tenor)):
        name = 'df' + tenor[i]
        df  = self.__dict__[name]
        dummy_list = self.basic_feature.copy()
        dum_data = pd.get_dummies(df,columns = dummy_list, drop_first=True)
        dum_data = self._add_interaction(dum_data, self.spec)

        hf = h2o.H2OFrame(dum_data)
        hf['Spread'].asnumeric()
        predictors = list(dum_data.columns)
        predictors = predictors.remove('Spread')
        response = "Spread"

        train, valid = hf.split_frame(ratios=[.8], seed=1234)
        encoding = 'OneHotExplicit'
        self.model.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid,
               categorical_encoding=encoding)
        pref = self.model.get_grid(sort_by ='rmse', decreasing=False )
        self.__dict__['model'+tenor[i]] = self.model.models[0]
    
    if self.model_type == "autoML":
      for i in range(len(tenor)):
        name = 'df' + tenor[i]
        df  = self.__dict__[name]

        dummy_list = self.basic_feature.copy()
        dum_data = pd.get_dummies(df,columns = dummy_list, drop_first=True)
        dum_data = self._add_interaction(dum_data,self.spec)
        hf = h2o.H2OFrame(dum_data)
        hf['Spread'].asnumeric()
        predictors = list(dum_data.columns)
        predictors = predictors.remove('Spread')
        response = "Spread"

        train, valid = hf.split_frame(ratios=[.8], seed=1234)
        self.model.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

  def model_predict(self):
    self._data_process()
    if self.model_type == "linear":
      
      dummy_list = self.basic_feature.copy()
      for i in range(len(self.tenor)):
        name = 'df' + self.tenor[i]
        df  = self.__dict__[name]
        
        dum_data = pd.get_dummies(df,columns = dummy_list, drop_first=True)
        dum_data = self._add_interaction(dum_data, self.spec)
        dum_data['Global'] = 1
        y = np.log(dum_data['Spread'])
        dum_data = dum_data.drop('Spread', axis = 1)
        dum_data.replace([np.inf, -np.inf], np.nan, inplace=True)
        dum_data = dum_data.dropna()
        self.model.fit(dum_data, y)
        
        pred = self.model.predict(dum_data)
        df_pred = pd.DataFrame(pred)
        df_pred.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/linear'+name+'.csv',index=False)
   
    if self.model_type == "random_forest":
      dummy_list = self.basic_feature.copy()
      for i in range(len(self.tenor)):
        name = 'df' + self.tenor[i]
        df  = self.__dict__[name]
        dum_all = pd.get_dummies(df,columns = dummy_list, drop_first=True)
        dum_all = self._add_interaction(dum_all, self.spec)
        hf = h2o.H2OFrame(dum_all)
        pred = self.model.predict(hf)
        df_pred = pred.as_data_frame()

        df_pred.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/rf'+name+'.csv',index=False)

    





  def USJP_prediction(self):

    import time
    import random
    if self.model_type == "random_forest":
      k = 10
      df = self.df5Y
      h2o.init()
      
      all_df = self.df5Y
      dummy_list = self.basic_feature.copy()
      dum_all = pd.get_dummies(all_df,columns = dummy_list, drop_first=True)
      dum_all = self._add_interaction(dum_all, self.spec)

      USJP_data = dum_all[(dum_all['Region_Japan'] == 1)| (dum_all['Region_United States'] == 1)]
      All_data = dum_all
      Other_data = dum_all[(dum_all['Region_Japan'] == 0) & (dum_all['Region_United States'] == 0)]

      spread_usjp = USJP_data['Spread']
      spread_all = All_data['Spread']
      spread_other = Other_data['Spread']

      hf_usjp = h2o.H2OFrame(USJP_data)
      hf_all = h2o.H2OFrame(All_data)
      hf_other = h2o.H2OFrame(Other_data)

      df_pred_usjp = pd.DataFrame()
      df_pred_all = pd.DataFrame()
      df_pred_other = pd.DataFrame()

      for i in range(1,k):
        t = int(time.time() * 1000.0)
        random.seed(((t & 0xff000000) >> 24) +((t & 0x00ff0000) >>8) +((t & 0x0000ff00) <<8) +((t & 0x000000ff) << 24))
        df['fold'] = np.random.randint(1, 5, df.shape[0])
        df_resample = df.loc[df['fold']!= 1]

        
        model =  H2ORandomForestEstimator(ntrees=100,max_depth=10)
        
        dum_data = pd.get_dummies(df_resample,columns = dummy_list, drop_first=True)
        dum_data = self._add_interaction(dum_data, self.spec)
        
        hf = h2o.H2OFrame(dum_data)
        hf['Spread'].asnumeric()
        predictors = list(dum_data.columns)
        predictors = predictors.remove('Spread')
        response = "Spread"
        print(hf)
        model.train(x=predictors,
               y=response,
               training_frame=hf)
  
        pred_usjp = model.predict(hf_usjp)
        pred_all = model.predict(hf_all)
        pred_other = model.predict(hf_other)

        temp1 = pred_usjp.as_data_frame() 
        temp2 = pred_all.as_data_frame() 
        temp3 = pred_other.as_data_frame()
        
        df_pred_usjp[i] = temp1
        df_pred_all[i] = temp2
        df_pred_other[i] = temp3

        
      
      
      df_pred_usjp.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/usjp.csv',index=False )
      df_pred_all.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/all.csv',index=False )
      df_pred_other.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/other.csv',index=False )
      spread_usjp.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/usjp_spread.csv',index=False )
      spread_all.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/all_spread.csv',index=False )
      spread_other.to_csv('/content/drive/MyDrive/Practicum/Week10/USJPresult/other_spread.csv',index=False )
      
      





  def sklearn_model_validation(self, cv_no, specific_tenor, spec):
    self.spec = spec
    self._data_process()
    
    if specific_tenor == 0:
      tenor = self.tenor
    else:
      tenor = specific_tenor

    if self.model_type == "linear":
      for i in range(len(tenor)):
        name = 'df' + tenor[i]
        df  = self.__dict__[name]

        sector_result = self._sklearn_partial_validation(df,'Sector',cv_no)
        region_result = self._sklearn_partial_validation(df,'Region',cv_no)
        rating_result = self._sklearn_partial_validation(df,'Rating',cv_no)
        result = sector_result + region_result + rating_result
        

        self.__dict__['result'+name] = pd.DataFrame()
        self.__dict__['result'+name]['Name'] = self.name_list
        self.__dict__['result'+name][self.evaluation_matrix] = result

        marker = [x[0].upper() for x in self.interaction_feature]
        marker = '_'.join(marker)
        path = self.result_export_path+name+'_'+ marker+'_' +'result.csv'
        self.__dict__['result'+name].to_csv(path, index=False)

  def _partial_validation(self,df,partial_name, cv_no):
    partial_result = []
    df_copy = df.copy()
    dummy_list = self.basic_feature.copy()
    dummy_list = dummy_list.remove(partial_name)
    for i in range(len(self.__dict__[partial_name+'_list'])):
      data = df_copy[df_copy[partial_name] == self.__dict__[partial_name+'_list'][i]]
      dum_data = pd.get_dummies(data,columns =  dummy_list, drop_first=True)
      
      #dum_data = dum_data.drop(str(partial_name), axis = 1)
      dum_data = self._add_interaction(dum_data, self.spec)
      dum_data['Global'] = 1


      temp = self._cross_validation(dum_data, cv_no)
      partial_result.append(temp)
    
    return partial_result
  
  def _sklearn_partial_validation(self,df,partial_name,cv_no):
    partial_result = []
    dummy_list = self.basic_feature.copy()
    dummy_list = dummy_list.remove(partial_name)
    for i in range(len(self.__dict__[partial_name+'_list'])):
      data = df[df[partial_name] == self.__dict__[partial_name+'_list'][i]]
      #data['Spread'] = data['Spread'].apply(np.log)
      dum_data = pd.get_dummies(data,columns =  dummy_list, drop_first=True)
      #print(dum_data)
      #dum_data = dum_data.drop(str(partial_name), axis = 1)
      dum_data = self._add_interaction(dum_data,self.spec)
      dum_data['Global'] = 1


      temp = self._sklearn_cross_validation(dum_data, cv_no)
      partial_result.append(temp)
    
    return partial_result






  def _list_same( self, a, b):
    same = [x for x in a if x in b]
    return same
  
  def _list_difference( self, a, b):
    same = [x for x in a if x in b]
    diff = [y for y in (a+b) if y not in same]
    return diff


  def _add_interaction(self, df, spec):
    #target is a list! First check if it is empty
    #will return a copy of dataframe

    df_copy = df.copy()
    if bool(self.interaction_feature):
      temp = df_copy.drop(self.interaction_feature, axis = 1)
      temp = temp.drop(self.label, axis = 1)
      temp_name = list(temp.columns)
      for i in range(len(temp_name)):
        for j in range(len(self.interaction_feature)):
          for k in range(len(spec)):
            if spec[k] in temp_name[i]:

              interact_name = temp_name[i] + '_' + self.interaction_feature[j]
              df_copy[interact_name] = df_copy[temp_name[i]] * df_copy[self.interaction_feature[j]]
          
    
    return df_copy
  
  def _min_max_scaling(self, df):
    # apply min-max scaling
    for column in self.interaction_feature:
      df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return df    

  def _get_velidation_list(self):
  #Using 5Y tenor data to get validation list
    df = self.df5Y
    self.Sector_list = df['Sector'].unique()
    self.Region_list = df['Region'].unique()
    self.Rating_list = df['Rating'].unique()
    self.Rating_list.sort()
    self.name_list = np.concatenate((self.Sector_list, self.Region_list, self.Rating_list))




  def _outlier_process(self, df):
  # dealing with outlier using cooks distance
  # We using linear regression model to fit with all features we have, then applying cooks distance to do residual
  # analysis and drop outlier

    outlier_index = []
    df_copy = df.copy()
    for i in range(len(self.Rating_list)):
      
      data = df_copy[df_copy['Rating'] == self.Rating_list[i]]
      data = pd.get_dummies(data,columns =['Sector','Region'], drop_first = True)
      data = data.drop(['Rating'] , axis = 1)
      data = self._add_interaction(data,['Sector', 'Region'])
      y = data[self.label].apply(np.log)
      x = data.drop(self.label, axis = 1)
      x['Global'] = 1

    
      # using basic linear model
      lm = sm.OLS(y, x).fit()
    
      ##Statistics
      #create instance of influence
      influence = lm.get_influence()
    
      #leverage (hat values)
      leverage = influence.hat_matrix_diag
  
      #Cook's D values (and p-values) as tuple of arrays
      cooks_d = influence.cooks_distance
      cook = cooks_d[0]
   
      index_cook = []
      firm_index = x.index
    
      #m = np.median(cook)
      for i in range(len(cook)):
        if cook[i] > 5/len(cook):
            index_cook.append(i)
    
      for i in range(len(index_cook)):
        outlier_index.append(firm_index[index_cook[i]])

    #drop outlier
    df_copy = df_copy.drop(outlier_index, axis = 0) 

    return df_copy  
  
  
  def _sklearn_cross_validation(self, df, no_cv):

    scoring = ['r2', 'neg_mean_squared_error','neg_root_mean_squared_error']
    
    y = df[self.label]
    x = df.drop(self.label, axis = 1)

    scores = cross_validate(self.model, x, y, cv=no_cv, scoring=scoring, return_train_score=True)

    result = []
    result.append(np.mean(-scores['test_neg_root_mean_squared_error']))
    result.append(np.mean(scores['train_r2']))
    result.append(np.mean(scores['test_r2']))
   

    return result

  
  def _cross_validation(self, df, no_cv):
    #input df should after dummy encoding
    #return is list

    self.n_splits = no_cv
    n = (df.shape[0] // self.n_splits)
    #should change as the number of matrix we return at _cv_algo
    avg_score = np.zeros(len(self.evaluation_matrix))
    
    for i in range(self.n_splits):
      #preparing the train and validation dataset
      validation = df[(i*n):((i+1)*n)]
      train = pd.concat([df[:(i*n)],df[((i+1)*n):]])
      if train.shape[0] > 4:
        if validation.shape[0] > 4:
          avg_score = list(np.array(avg_score) + np.array(self._cv_algo(train ,validation)))
    
    #print(avg_score)
    avg_score = list(np.array(avg_score)/no_cv)

    return avg_score
  
  def _cv_algo(self, train, validation):
    

    train_y = train[self.label]
    train_x = train.drop(self.label, axis = 1)

    validation_y = validation[self.label]
    validation_x = validation.drop(self.label, axis = 1)


    if self.model_type == "linear":
      self.model.fit(train_x, train_y)
      pred_y = self.model.predict(validation_x)
      pred_y = pred_y.round(5)

      #consider log transform bias
      temp = np.exp(validation_y - pred_y)
      temp = temp[np.isfinite(temp)]
      bias = sum(temp)/len(temp)
      retrans_y = np.exp(pred_y)*bias 
      retrans_y[np.isinf(retrans_y)] = 1000
      '''
      #debug use
      mae = metrics.mean_absolute_error(validation_y, pred_y)
      mse = metrics.mean_squared_error(validation_y, pred_y)
      rmse = np.sqrt(mse)
      train_R2 = self.model.score(train_x, train_y)
      test_R2 = self.model.score(validation_x, validation_y)
      
      train_adjR2 = 1 - (1-self.model.score(train_x, train_y))*(len(train_y)-1)/(len(train_y)-train_x.shape[1]-1)
      test_adjR2 = 1 - (1-self.model.score(validation_x, validation_y))*(len(validation_y)-1)/(len(validation_y)-validation_x.shape[1]-1)

      '''
      #All matrix 
      mae = metrics.mean_absolute_error(np.exp(validation_y), retrans_y)
      mse = metrics.mean_squared_error(np.exp(validation_y), retrans_y)
      rmse = np.sqrt(mse)
      #R square
      train_R2 = self.model.score(train_x, train_y)
      test_R2 = self.model.score(validation_x, validation_y)
      
      train_adjR2 = 1 - (1-self.model.score(train_x, train_y))*(len(train_y)-1)/(len(train_y)-train_x.shape[1]-1)
      test_adjR2 = 1 - (1-self.model.score(validation_x, validation_y))*(len(validation_y)-1)/(len(validation_y)-validation_x.shape[1]-1)
      
      matrix = [rmse, train_adjR2, test_adjR2]
    return matrix

  def _dummy_encoding(self, df, dummy_list):
    df = pd.get_dummies(df, columns = dummy_list, drop_first = True)

    return df
  
  def _clean_dataset(self,df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
  







In [ ]:
test2 = model(test,['Sector',
 'Region',
 'Rating',
 'return_range',
 'Volatility_1m','Spread'],'linear')

In [ ]:
test2.sklearn_model_validation(3,0, ['Rating','Region'])

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:695: RuntimeWarning: invalid value encountered in sqrt
  return self.resid / sigma / np.sqrt(1 - hii)
/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:695: RuntimeWarning: divide by zero encountered in true_divide
  return self.resid / sigma / np.sqrt(1 - hii)
/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:708: RuntimeWarning: divide by zero encountered in true_divide
  cooks_d2 *= hii / (1 - hii)


In [ ]:
test2.df5Y['Spread']

0        1.804031
12       1.722392
19       1.325284
30       1.324915
52       1.439578
           ...   
15817    1.641908
15829    1.607209
15843    1.407923
15856    1.239709
15869    1.347461
Name: Spread, Length: 1199, dtype: float64

In [ ]:
test3 = model(test,['Sector',
 'Region',
 'Rating',
 'return_range',
 'Volatility_1m','Spread'],'random_forest')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,4 hours 21 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_unknownUser_f2wx86
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.098 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
#test3.model_validation(3,'5Y',[])
test3.model_validation(3,'5Y',['Rating','Region'])

/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:695: RuntimeWarning: invalid value encountered in sqrt
  return self.resid / sigma / np.sqrt(1 - hii)
/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:695: RuntimeWarning: divide by zero encountered in true_divide
  return self.resid / sigma / np.sqrt(1 - hii)
/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:708: RuntimeWarning: divide by zero encountered in true_divide
  cooks_d2 *= hii / (1 - hii)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:
test3.df5Y['Spread']

0        1.804031
12       1.722392
19       1.325284
30       1.324915
52       1.439578
           ...   
15817    1.641908
15829    1.607209
15843    1.407923
15856    1.239709
15869    1.347461
Name: Spread, Length: 1201, dtype: float64

In [ ]:
test3.USJP_prediction()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 41 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_unknownUser_f2wx86
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.206 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


return_range,Volatility_1m,Spread,fold,Sector_Consumer Goods,Sector_Consumer Services,Sector_Energy,Sector_Financials,Sector_Healthcare,Sector_Industrials,Sector_Technology,Sector_Telecommunications Services,Sector_Utilities,Region_Europe,Region_Japan,Region_North America,Region_United States,Region_others,Rating_AA,Rating_B,Rating_BB,Rating_BBB,Region_Europe_return_range,Region_Europe_Volatility_1m,Region_Japan_return_range,Region_Japan_Volatility_1m,Region_North America_return_range,Region_North America_Volatility_1m,Region_United States_return_range,Region_United States_Volatility_1m,Region_others_return_range,Region_others_Volatility_1m,Rating_AA_return_range,Rating_AA_Volatility_1m,Rating_B_return_range,Rating_B_Volatility_1m,Rating_BB_return_range,Rating_BB_Volatility_1m,Rating_BBB_return_range,Rating_BBB_Volatility_1m
0.428571,0.00824285,6.07408,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.428571,0.00824285,0,0,0,0,0.428571,0.00824285,0,0,0,0
0.206349,0.0145644,5.59791,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.206349,0.0145644,0,0
0.214278,0.0181372,3.76325,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.214278,0.0181372,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0.214278,0.0179518,3.76187,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.214278,0.0179518,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0.276674,0.0225686,4.21891,3,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0.276674,0.0225686,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.276674,0.0225686
0.131241,0.0274926,4.55063,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.131241,0.0274926,0,0,0,0,0,0,0.131241,0.0274926
0.356866,0.0392584,6.08433,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0.356866,0.0392584,0,0,0.356866,0.0392584,0,0,0,0
0.356866,0.0392584,5.70174,2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.356866,0.0392584,0,0,0,0,0,0,0,0,0,0,0,0,0.356866,0.0392584,0,0
0.125094,0.0148333,4.58019,3,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.125094,0.0148333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.125094,0.0148333
0.490061,0.0383045,5.96054,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.490061,0.0383045,0,0,0,0,0.490061,0.0383045,0,0,0,0



drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |

...


drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


return_range,Volatility_1m,Spread,fold,Sector_Consumer Goods,Sector_Consumer Services,Sector_Energy,Sector_Financials,Sector_Healthcare,Sector_Industrials,Sector_Technology,Sector_Telecommunications Services,Sector_Utilities,Region_Europe,Region_Japan,Region_North America,Region_United States,Region_others,Rating_AA,Rating_B,Rating_BB,Rating_BBB,Region_Europe_return_range,Region_Europe_Volatility_1m,Region_Japan_return_range,Region_Japan_Volatility_1m,Region_North America_return_range,Region_North America_Volatility_1m,Region_United States_return_range,Region_United States_Volatility_1m,Region_others_return_range,Region_others_Volatility_1m,Rating_AA_return_range,Rating_AA_Volatility_1m,Rating_B_return_range,Rating_B_Volatility_1m,Rating_BB_return_range,Rating_BB_Volatility_1m,Rating_BBB_return_range,Rating_BBB_Volatility_1m
0.428571,0.00824285,6.07408,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.428571,0.00824285,0,0,0,0,0.428571,0.00824285,0,0,0,0
0.206349,0.0145644,5.59791,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.206349,0.0145644,0,0
0.214278,0.0179518,3.76187,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.214278,0.0179518,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0.276674,0.0225686,4.21891,4,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0.276674,0.0225686,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.276674,0.0225686
0.170814,0.0284664,5.15507,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.170814,0.0284664,0,0
0.131241,0.0274926,4.55063,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.131241,0.0274926,0,0,0,0,0,0,0.131241,0.0274926
0.356866,0.0392584,5.70174,4,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.356866,0.0392584,0,0,0,0,0,0,0,0,0,0,0,0,0.356866,0.0392584,0,0
0.125094,0.0148333,4.58019,3,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.125094,0.0148333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.125094,0.0148333
0.282552,0.0233139,4.57615,4,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0.282552,0.0233139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.282552,0.0233139
0.490061,0.0383045,5.96054,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.490061,0.0383045,0,0,0,0,0.490061,0.0383045,0,0,0,0



drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


return_range,Volatility_1m,Spread,fold,Sector_Consumer Goods,Sector_Consumer Services,Sector_Energy,Sector_Financials,Sector_Healthcare,Sector_Industrials,Sector_Technology,Sector_Telecommunications Services,Sector_Utilities,Region_Europe,Region_Japan,Region_North America,Region_United States,Region_others,Rating_AA,Rating_B,Rating_BB,Rating_BBB,Region_Europe_return_range,Region_Europe_Volatility_1m,Region_Japan_return_range,Region_Japan_Volatility_1m,Region_North America_return_range,Region_North America_Volatility_1m,Region_United States_return_range,Region_United States_Volatility_1m,Region_others_return_range,Region_others_Volatility_1m,Rating_AA_return_range,Rating_AA_Volatility_1m,Rating_B_return_range,Rating_B_Volatility_1m,Rating_BB_return_range,Rating_BB_Volatility_1m,Rating_BBB_return_range,Rating_BBB_Volatility_1m
0.428571,0.00824285,6.07408,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.428571,0.00824285,0,0,0,0,0.428571,0.00824285,0,0,0,0
0.206349,0.0145644,5.59791,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.206349,0.0145644,0,0
0.214278,0.0179518,3.76187,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.214278,0.0179518,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0.170814,0.0284664,5.15507,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.170814,0.0284664,0,0
0.131241,0.0274926,4.55063,3,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.131241,0.0274926,0,0,0,0,0,0,0.131241,0.0274926
0.356866,0.0392584,6.08433,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0.356866,0.0392584,0,0,0.356866,0.0392584,0,0,0,0
0.356866,0.0392584,5.70174,3,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.356866,0.0392584,0,0,0,0,0,0,0,0,0,0,0,0,0.356866,0.0392584,0,0
0.282552,0.0233139,4.57615,2,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0.282552,0.0233139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.282552,0.0233139
0.20821,0.0226142,3.82619,4,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.20821,0.0226142,0,0,0,0,0,0,0,0,0,0
0.148342,0.0169649,4.23624,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.148342,0.0169649



drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [ ]:

pref = test3.model.get_grid(sort_by ='rmse', decreasing=False )
pref.sorted_metric_table()

,,max_depth,ntrees,model_ids,rmse
0,,15.0,30.0,RFR_grid_model_213,0.031202
1,,10.0,100.0,RFR_grid_model_236,0.031397
2,,10.0,90.0,RFR_grid_model_230,0.031437
3,,10.0,50.0,RFR_grid_model_218,0.031674
4,,30.0,70.0,RFR_grid_model_227,0.031690
...,...,...,...,...,...
235,,5.0,30.0,RFR_grid_model_121,0.200850
236,,5.0,100.0,RFR_grid_model_25,0.200946
237,,40.0,50.0,RFR_grid_model_72,0.201021
238,,10.0,30.0,RFR_grid_model_2,0.201194


In [ ]:
hf = h2o.H2OFrame(test.df5Y)
pred = test3.model5Y.predict(hf)
pred

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column 'Sector_Consumer Goods': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column 'Sector_Consumer Services': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column 'Sector_Energy': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column 'Sector_Financials': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset is missing column 'Sector_Healthcare': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation datas

predict
4.33576
4.27656
4.28001
4.282
4.34047
4.34336
4.27335
4.27629
4.27669
4.34369


In [ ]:
'''
test4 = model(test,['Sector',
 'Region',
 'Rating',
 'Volatility_1m','return_range','Spread'],'autoML')

test4.model_validation(3,'5Y')
'''

"\ntest4 = model(test,['Sector',\n 'Region',\n 'Rating',\n 'Volatility_1m','return_range','Spread'],'autoML')\n\ntest4.model_validation(3,'5Y')\n"

In [ ]:
test4.model.leaderboard

NameError: ignored

In [ ]:
k = pd.get_dummies(test3.df5Y, columns = ['Region','Rating'], drop_first=True)
k

In [ ]:
test3.Sector_list
data = test3.df5Y[test3.df5Y['Sector'] == test3.Sector_list[3]]
data = pd.get_dummies(data, columns = ['Region','Rating'], drop_first=True)
data

1. data set tenor split: Df.1Y? have a loop function to call every tenor, data processing -  best feature choose, preprocess --- change datafreame contain, add E2C, export data
4. Model:(feature, data, modelname) feature needed: add interaction(generate interatcion list), normalization, outlier, velidation(get list, cross_validation for three feature(validation append method, then do three time) and append result), self_test method

